# Predicting Pet Insurance Claims - EDA

## 1 Introduction

### 1.1 Background
Whenever a pet insurance policy holder incurs veterinary expenses related to their enrolled pet, they can submit claims for reimbursement, and the insurance company reimburses eligible expenses. To price insurance products correctly, the insurance company needs to have a good idea of the amount their policy holders are likely to claim in the future. 

### 1.2 Project Goal
The goal of this project is to create a machine learning model to predict how much (in dollars) a given policy holder will claim for during the second year of their policy. 

### 1.3 Initial Questions for EDA
Below are a few initial questions to answer and areas of interest for the detailed data analysis.
* What is the distribution of our claims amounts by year?
    * What, if anything, is needed to deal with the very high (claims > \\$10k) and very low (claims = \$0) claims amounts?
* What patterns or relationships exist between 'Species' and amounts claimed?
* What patterns or relationships exist between 'Breed' and amounts claimed?
* Does EnrollPath factor in to claims amounts in a meaningful way or should it be dropped?
* Is there a connection between PetAge and number or amount of claims?

# 2 Setup

## 2.1 Imports

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# import random

import matplotlib.pyplot as plt
# %matplotlib inline
import seaborn as sns

## 2.2 Data Load & Preview

In [2]:
# Read in the data file generated during data wrangling
df = pd.read_csv('../data/merged_pets.csv')

# Preview
df.head()

,PetId,Species,Breed,Premium,Deductible,EnrollPath,AgeYr1,YoungAge,MixedBreed,AmtClaimsYr1,AmtClaimsYr2,AvgClaimsYr1,AvgClaimsYr2,NumClaimsYr1,NumClaimsYr2
0,0,Dog,Schnauzer Standard,84.54,200,Web,3,0,0,0.00,1242.00,0.000000,621.000,0,2
1,1,Dog,Yorkiepoo,50.33,500,Phone,0,0,0,0.00,0.00,0.000000,0.000,0,0
2,2,Dog,Mixed Breed Medium,74.00,500,Phone,0,0,1,640.63,1187.68,213.543333,237.536,3,5
3,3,Dog,Labrador Retriever,57.54,500,Phone,0,0,0,0.00,0.00,0.000000,0.000,0,0
4,4,Dog,French Bulldog,60.69,700,Web,0,0,0,7212.25,168.75,801.361111,168.750,9,1


## 3 Claims - What is the Distribution of Claims?
In the data wrangling notebook, we looked at the claims data with each claim as its own data point. We observed some very high claims amounts (> \\$10,000), but we didn't do any deeper analysis to decide what to do about the outlier values. As a next step, let's view the distribution of claims for year 1 and year 2, respectively, and decide what to do with the outliers.

NOTES: How to handle the outliers?
* Drop rows above a certain threshold
* Cap total amount at a certain threshold(t) (i.e., if x > t, then x = t)
* If capping, should the amount removed be somehow factored back into the data?
* Drop pets with greater than *n* high dollar claims, where *n* is some predetermined threshold

BREED:
* Does it make sense to look at the proportion of a breed with high claims and see if any of the breeds stand out?
* Should I only pay attention to this is the breeds in question have a count of pets higher than some significant number (e.g., 30)?

### 3.1 Distribution of total claims amounts for years 1 and 2

In [34]:
# Review basic stats for claims in years 1 and 2
df[['AmtClaimsYr1', 'AmtClaimsYr2']].describe()

,AmtClaimsYr1,AmtClaimsYr2
count,50000.000000,50000.000000
mean,797.504936,691.911630
std,2168.235494,2242.769687
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,656.547500,412.212500
max,61770.470000,70251.600000


In [40]:
# Check the 99th percentile of claims totals for each year
print("99% of Pets had total claims in year 1 less than " + str(round(df.AmtClaimsYr1.quantile(.99), 4)))
print("99% of Pets had total claims in year 2 less than " + str(round(df.AmtClaimsYr2.quantile(.99), 4)))

99% of Pets had total claims in year 1 less than 10685.7036
99% of Pets had total claims in year 2 less than 10490.7582


Looking at the distribution data, it's clear there is a small number of pets in each year with very high total claims amounts. The rollup is helpful, but it does obscure some of the details about the underlying claims.

Before we decide what to do about the pets with outlier claims totals in year 1 and 2, let's take a step back and look into the individual claims amounts.

#### 3.1.1 Review data for individual claims 

In [44]:
# Read in the cleaned claims data
claims = pd.read_csv('../data/cleaned_claims.csv', index_col=0)
claims.head()

,PetId,ClaimDate,AmountClaimed,EnrollDate,Year
0,0,2020-03-07 12:17:00,624.00,2018-09-18 17:51:00,Yr2
1,0,2020-03-07 08:03:00,618.00,2018-09-18 17:51:00,Yr2
2,2,2020-02-08 00:18:00,199.52,2018-08-22 11:23:00,Yr2
3,2,2018-09-21 14:34:00,199.60,2018-08-22 11:23:00,Yr1
4,2,2018-10-23 01:40:00,162.40,2018-08-22 11:23:00,Yr1


In [45]:
# Review the descriptive statistics
claims.AmountClaimed.describe()

count    154889.000000
mean        480.801273
std         942.549867
min           0.190000
25%          95.000000
50%         201.200000
75%         445.860000
max       45084.990000
Name: AmountClaimed, dtype: float64

In [48]:
# Calculate the 99th percentile of claims amounts
pct_99 = round(np.percentile(claims.AmountClaimed, 99), 2)
print("99% of claims are below " + str(pct_99))

99% of claims are below 4899.08


In [49]:
# Filter to rows with high claims 
high_claims = claims[claims.AmountClaimed >= pct_99]

# Groupby PetId and get a count of expensive claims for each pet
exp_pets = high_claims.groupby('PetId').agg({'PetId': 'count'})

# Check to see how many pets have more than one high dollar claim
exp_pets.PetId.value_counts()

1    824
2    235
3     53
4      9
5      9
7      1
8      1
Name: PetId, dtype: int64

In [ ]:
np.percentile(df.AmtClaimsYr1, 99.8)

In [ ]:
np.percentile(df.AmtClaimsYr2, 99.8)

In [ ]:
df[df.AmtClaimsYr1 > 18304].Species.value_counts()

In [ ]:
df[df.AmtClaimsYr1 > 18304].Breed.value_counts()

In [ ]:
df[df.AmtClaimsYr2 > 20828].Species.value_counts()

In [ ]:
df[df.AmtClaimsYr2 > 18304].Breed.value_counts()

In [ ]:
# Create histograms showing the distribution of claims for years 1 and 2 
fig, (ax1, ax2) = plt.subplots(1,2, sharey=True, figsize=(20,6))
ax1.hist(df.AmtClaimsYr1, bins=20)
ax1.set_title("Claims Distribution, Year 1")
ax2.hist(df.AmtClaimsYr2, bins=20)
ax2.set_title("Claims Distribution, Year 2")
plt.show()

Obervations - lots of \\$0 claims

In [ ]:
# Filter out $0 Claims Amounts
yr1_greater_0 = df[df.AmtClaimsYr1 > 0]['AmtClaimsYr1'] 
yr2_greater_0 = df[df.AmtClaimsYr2 > 0]['AmtClaimsYr2'] 

# Create histograms showing the distribution of claims for years 1 and 2 
fig, (ax1, ax2) = plt.subplots(1,2, sharey=True, figsize=(20,6))
ax1.hist(yr1_greater_0, bins=20)
ax1.set_title("Claims Distribution, Year 1")
ax2.hist(yr2_greater_0, bins=20)
ax2.set_title("Claims Distribution, Year 2")
plt.show()

## 4 Species - Are Dogs More Expensive Than Cats?  
As the current owner of two cats and the former owner of two dogs, this seems like a pretty easy question to answer for me. Based on my own experiences, dogs are much more expensive than cats in about every dimension possible. They are larger and tend to eat more (much more in some cases), they typically spend more of their lives outside making them more prone to injury and they go through chew toys like crazy! But, do dog owners spend more on vet bills than cat owners?

### 4.1 Aggregate data by species

Rather than jumping straight to claims amounts, let's instead look at a breakdown of all the data by species to see if we see anything of interest.

In [ ]:
# Group df by 'Species' and view column means
df.groupby(by='Species').agg('mean')

In [ ]:
# Group df by 'Species' and view column means
df.groupby(by='Species').agg('median')

#### 4.1.1 Initial observations
Some initial observations based on the breakdown above:
1. On average, Dogs have higher premiums and higher deductibles
2. Dogs tend to be enrolled at a younger age by nearly a year on average
3. The averages for claims data (i.e., number of claims, average claim amount, and total claims amount) is higher for dogs across the board and in some cases, is as much as 2x higher than for cats.
4. While not a difference between cats and dogs, it's interesting to see that the median value for all claims categories is 0. This last point will almost certainly factor in as we move forward with analysis.

### 4.2 Claims differences between species
Since the focus of this study is on claims amounts, we'll focus on claims-related categories for now. The difference in the average claims amounts between cats and dogs in year one appears to be significant with dogs (\\$867) at more than 2x higher than cats (\\$433). And in year 2, the average for dogs (\\$739) is not quite 2x the average for cats (\\$445), but it's close.

Based on this, we may need to factor this in to our predictive model later on as it appears that a single model may not work well across the two species.

Before we make the final decision, let's plot the data for a better look.

#### 4.2.1 Disstribution of claims by species

In [ ]:
# Create boxplot showing the distribution of claims totals by species
amt_claims_total = df["AmtClaimsYr1"] + df["AmtClaimsYr2"]
sns.catplot(x="Species", y=amt_claims_total, data=df, kind="box", sym="")
plt.ylabel("Total Claims Amount (outliers removed)")
plt.title("Total Claims by Species")
plt.show()

The above plot tells a pretty compelling story. After removing outliers, the entire distribution of claims totals for cats fits within the first 75% of the distribution for dogs. And the remaining 25% of claims totals for dogs covers a spread nearly twice as big as the entire range for cats. 

Let's plot another view for a different perspective.

#### 4.2.2 Total number of claims vs. total amount of claims

In [ ]:
# Calculate total number of claims per pet
num_claims_total = df["NumClaimsYr1"] + df["NumClaimsYr2"]

# Create a scatter plot showing number and amount of claims by species
plt.figure(figsize=(10,6))
_ = sns.scatterplot(x=num_claims_total, y=amt_claims_total, data=df, hue='Species')

plt.xlabel("Number of Claims")
plt.ylabel("Amount of Claims")
plt.title("Total Claims by Species")

legend = plt.legend(title="", fancybox=True)

plt.show()

Based on the above, it seems the overall story may not be straightforward. The scatter plot above shows quite a bit of variability in the data regardless of species, with no clear way to classify a pet into one species or another based solely on number and amount of claims.

Before we move on, let's make one additional plot to help us better understand the breakdown of the data.

#### 4.2.3 Total claims amount by species and age

In [ ]:
# Create a nested barplot 
sns.set_style("whitegrid")
_ = sns.catplot(kind="bar", x="AgeYr1", y=amt_claims_total, data=df, hue="Species", aspect=2, height=6)

_.set_axis_labels("Age at Enrollment", "Total Claims")
_.despine(left=True)
_.legend.set_title("")

This is an interesting perspective. Claims for dogs exceed claims for cats by a considerable margin at every age along the spectrum with the exception of the oldest age in our dataset (13). In addition, the confidence interval on the claims data is generally increasing in proportion to pet age. Intuitively this makes sense given that we would expect to see a greater variance in healthcare costs as pets age (similar to what is observed in people).

### 4.3 Conclusions about species
* Species does appear to play a significant factor in claims amounts and will need to be taken into account as we build out the predictive model.
* Number and amount of claims is highly variable regardless of species.
* Pet age also appears to be a factor relating to claims amount, and a bit more so for dogs than cats until roughly age 10. The variability of claims amounts definitely increases with age and we should look into this further.


## 5 Breed - Does Breed Factor in to Claims Amounts?

TBD - write up intro for this section with initial list of questions

### 5.1 Cat breeds
TBD - write up if needed

Since we only have ~35 total cat breeds, we can take a quick look at a rollup of claims data for all breeds to help guide next steps.

In [ ]:
# Create cats dataframe
cats = df[df.Species == 'Cat'].copy()

# Group cats by breed and aggregate claims data columns
cats_by_breed = cats.groupby('Breed').agg({'PetId': ['count'],
                                           'NumClaimsYr1':['mean'],
                                           'NumClaimsYr2':['mean'],
                                           'AmtClaimsYr1':['mean', 'median', 'min', 'max'], 
                                           'AmtClaimsYr2':['mean', 'median', 'min', 'max']}
                                         )
# View result
cats_by_breed

#### 5.1.1 Initial observations on cat breeds
TBD
* The number of cats for each breed covers a wide range, from 1 pet for certain breeds to 3511 for the most common breed. 
* Some breeds with a low count have 0 claims (e.g., American Wirehair and Chartreux), while others have very high claims (e.g., Selkirk Rex). 
* The most common values for median and minimum claims amounts is \\$0. This matches up with what we observed earlier on in the analysis and will likely factor in to the final predictions.
* Others??

TODO for conclusions - will need to pick a count that is reasonable and then group breeds below that count into an "Other" category.

In [ ]:
# Preview number of cat breeds with a count of pets greater than 30
cats_by_breed[cats_by_breed['PetId']['count'] >= 30]['PetId']['count'].sort_values(ascending=False)

In [ ]:
# Preview full result
cats_by_breed[cats_by_breed['PetId']['count'] >= 30].sort_values([('AmtClaimsYr2', 'mean')], ascending=False)

In [ ]:
# Preserve list of Breeds with count greater equal to 30
cat_breeds = cats_by_breed[cats_by_breed['PetId']['count'] >= 30].index.to_list()

# cats_by_breed.reset_index(inplace=True)

# Group breeds as 'Other' if count less than 30


In [ ]:
# x = cats_by_breed[cats_by_breed['PetId']['count'] >= 30].index
# y1 = cats_by_breed[cats_by_breed['PetId']['count'] >= 30][('AmtClaimsYr1', 'mean')]
# y2 = cats_by_breed[cats_by_breed['PetId']['count'] >= 30][('AmtClaimsYr2', 'mean')]

plt.figure(figsize=(10,6))
# _ = plt.barh([y1,y2], stacked=True, width=1.0)
cats_by_breed[cats_by_breed['PetId']['count'] >= 30][[('AmtClaimsYr1', 'mean'),('AmtClaimsYr2', 'mean')]].plot(kind='barh', stacked=True)

plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Preview full result
cats_by_breed[cats_by_breed['PetId']['count'] < 30].sort_values([('AmtClaimsYr2', 'mean')], ascending=False)

In [ ]:
###########
# TODO
##########
# View distribution of average claims by cat breed (below seems like it would work for dogs as well)
## Maybe a scatter plot - each breed is a point plotted based on avg total num claims and avg total amount claims 
## Scatter plot circle size based on count of breed

# Remove outlier claims amounts
# Group cat breeds with fewer than 20 or 30 representative cats (unless it seems there is a significant difference maybe?? how to do this?)


In [ ]:
breed_counts = cats_by_breed['NumClaimsTotal', 'count']
breed_num_claims = cats_by_breed['NumClaimsTotal', 'mean']
breed_amt_claims = cats_by_breed['AmtClaimsTotal', 'mean']

plt.figure(figsize=(10,6))
_ = sns.scatterplot(data=cats_by_breed, x=breed_num_claims, y=breed_amt_claims, 
                    size=breed_counts, sizes=(20, 200), legend=False)
plt.show()

In [ ]:
cats_by_breed = cats_by_breed[cats_by_breed['AmtClaimsTotal', 'mean'] < 2000]

In [ ]:
breed_counts = cats_by_breed['NumClaimsTotal', 'count']
breed_num_claims = cats_by_breed['NumClaimsTotal', 'mean']
breed_amt_claims = cats_by_breed['AmtClaimsTotal', 'mean']

plt.figure(figsize=(10,6))
_ = sns.scatterplot(data=cats_by_breed, x=breed_num_claims, y=breed_amt_claims, 
                    hue=breed_counts, size=breed_counts, sizes=(20, 200), legend=True)

plt.xlabel('Avg Total Number of Claims')
plt.ylabel('Avg Total Amount of Claims')
plt.title('Average Claims by Breed')
# _._legend.set_title('Count of Breed')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.5, title="Count of Breed")

plt.show()

## Consider plotting the claims in year 1 and 2 after removing the outlier data to get a better distribution.

## Correlation between Yr2 claims and other features

In [ ]:
cats_corr = cats.corr()
# dogs_corr = dogs.corr()

In [ ]:
cats_corr

Total and average claims are highly correlated with yr1 and yr2 totals and averages, respectively. This makes sense. Some additional areas for analysis include:

1. Investigate to see if high yr1 claims tend to lead to high yr2 claims, or if there is any connection. (correlation is only 0.177)

2. 

In [ ]:
plt.figure(figsize=(10,6))
_ = sns.heatmap(cats_corr)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
_ = sns.heatmap(dogs_corr)
plt.show()

In [ ]:
# Review Distributions of all numeric columns
df.hist(figsize=(15,10))
_ = plt.subplots_adjust(hspace=0.5)
plt.show()

## Are claims in year 1 a good predictor of claims in year 2? 

In [ ]:
sns.scatterplot(x="AmtClaimsYr1", y="AmtClaimsYr2", data=df)
plt.show()

In [ ]:
# Linear regression

# Create arrays of values
AmtClaimsYr1 = df.AmtClaimsYr1.values
AmtClaimsYr2 = df.AmtClaimsYr2.values

# Create the scatterplot
plt.figure(figsize=(10,6))
# _ = plt.plot(AmtClaimsYr1, AmtClaimsYr2, marker='.', linestyle='none')
sns.scatterplot(x="AmtClaimsYr1", y="AmtClaimsYr2", data=df)
plt.margins(0.02)
_ = plt.xlabel('Claims Total in Year 1')
_ = plt.ylabel('Claims Total in Year 2')

# Perform a linear regression using np.polyfit(): a, b
a, b = np.polyfit(AmtClaimsYr1, AmtClaimsYr2, 1)

# # Print the results to the screen
# print('slope =', a, 'Claims Total in Year 2 / Claims Total in Year 1')
# print('intercept =', b, 'Claims Total in Year 2')

# Make theoretical line to plot
x = np.array([0, 70000])
y = a * x + b

# Add regression line to your plot
_ = plt.plot(x, y, color='orange')

# Draw the plot
plt.show()

In [ ]:
######
# TODO - Consider doing a similar plot as above for yr1 vs total and year2 vs total? Or a facet grid of all features
######

## What percentage of pets have no claims at all? Claims in year 1 and in year 2? Claims in only one of the two years?

In [ ]:
# Calculate the percentages of pets with claims > $0 in 1 yr, 2 yrs and not at all 
no_claims = round(df[(df['AmtClaimsYr1'] == 0) & (df['AmtClaimsYr2'] == 0)].shape[0] / df.shape[0] * 100)
both_yr_claims = round(df[(df['AmtClaimsYr1'] > 0) & (df['AmtClaimsYr2'] > 0)].shape[0] / df.shape[0] * 100)
one_yr_claims = 100 - (no_claims + both_yr_claims)

# Print the result
print(str(no_claims) + "% of pets have no claims")
print(str(one_yr_claims) + "% of pets have claims in only one year")
print(str(both_yr_claims) + "% of pets have claims in both years")

## Is there a connection between the number of claims submitted in year 1 and year 2?

## Breed analysis

In [ ]:
# Groupby Breed column and aggregate the mean of the total claims amount
breed_claims = df.groupby(by=['Species', 'Breed']).AmtClaimsTotal.agg(['mean', 'count']).sort_values(by='mean',
                                                                                        ascending=False)
# Rename aggregate columns 
breed_claims.rename(columns={"mean": "AvgTotalClaims", "count": "BreedCount"}, inplace=True)

# Preview descriptive statistics 
breed_claims.describe()

In [ ]:
breed_claims.head(50)

In [ ]:
breed_claims.tail(20)

In [ ]:
cond_plot = sns.FacetGrid(data=cats, col='Breed', hue='CentralAir', col_wrap=4)
cond_plot.map(sns.scatterplot, 'OverallQual', 'SalePrice');

## EnrollPath - Do people who enroll on the web submit more claims on average than those enrolling by phone?

Does enrollpath matter in terms of number of claims filed?
Look at median claims

In [ ]:
# Preview enroll path entries and value counts
df.EnrollPath.value_counts()

In [ ]:
# Group data by enrollpath
enrollpath = df.groupby(by='EnrollPath')

# Display aggregated statistics for total claims amount by enrollpath
enrollpath.NumClaimsTotal.agg(['mean', 'median', 'min', 'max'])

In [ ]:
# Display aggregated statistics for total claims amount by enrollpath
enrollpath.AmtClaimsTotal.agg(['mean', 'median', 'min', 'max'])

## How much does age play a roll in yr2 claims totals?
1. Look at average claims amount grouped by age in years
2. Scatterplot of AgeYr2 and AmtClaimsYr2 (maybe do this all up and also a facetgrid with a plot per year)
3. 

In [ ]:
# Aggregate mean claims data per year and total
yr1_claims = df.groupby(by='AgeYr2').AmtClaimsYr1.agg('mean')
yr2_claims = df.groupby(by='AgeYr2').AmtClaimsYr2.agg('mean')
total_claims = df.groupby(by='AgeYr2').AmtClaimsTotal.agg('mean')

# Create plots and label
plt.figure(figsize=(10,6))
_ = plt.plot(yr1_claims, label='Year 1 Claims')
_ = plt.plot(yr2_claims, label='Year 2 Claims')
_ = plt.plot(total_claims, label='Total Claims')

plt.xlabel('Pet Age in Years')
plt.ylabel('Dollars')

plt.legend()
plt.show()

In [ ]:
# # Isolate rows with high premiums for additional review
# high_prems = pets[pets['Premium'] > 200]

# # Review date distribution
# plt.figure(figsize=(6, 4))
# plt.title('Distribution of High Premiums')
# plt.hist(high_prems.Premium, bins = 20)
# plt.show()

In [ ]:
# # Review the number of pets with high premiums by species
# high_prems.Species.value_counts()

In [ ]:
# Plotting a histogram
# Compute number of data points: n_data
n_data = len(versicolor_petal_length)

# Number of bins is the square root of number of data points: n_bins
n_bins = np.sqrt(n_data)

# Convert number of bins to integer: n_bins
n_bins = int(n_bins)

# Plot the histogram
_ = plt.hist(versicolor_petal_length, bins=n_bins)

# Label axes
_ = plt.xlabel('petal length (cm)')
_ = plt.ylabel('count')

# Show histogram
plt.show()

In [ ]:
# Plotting a beeswarm plot
# Create bee swarm plot with Seaborn's default settings
_ = sns.swarmplot(x='species', y='petal length (cm)', data=df)

# Label the axes
_ = plt.xlabel('species')
_ = plt.ylabel('petal length (cm)')

# Show the plot
plt.show()

In [ ]:
# ECDF function
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    # Number of data points: n
    n = len(data)

    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y

In [ ]:
# Compute and plot ECDF 

# Compute ECDF for versicolor data: x_vers, y_vers
x_vers, y_vers = ecdf(versicolor_petal_length)

# Generate plot
_ = plt.plot(x_vers, y_vers, marker='.', linestyle='none')

# Label the axes
_ = plt.xlabel('petal length (cm)')
_ = plt.ylabel('ECDF')

# Display the plot
plt.show()

In [ ]:
# Comparison of ECDFs

# Compute ECDFs
x_set, y_set = ecdf(setosa_petal_length)
x_vers, y_vers = ecdf(versicolor_petal_length)
x_virg, y_virg = ecdf(virginica_petal_length)

# Plot all ECDFs on the same plot
_ = plt.plot(x_set, y_set, marker='.', linestyle='none')
_ = plt.plot(x_vers, y_vers, marker='.',linestyle='none')
_ = plt.plot(x_virg, y_virg, marker='.',linestyle='none')

# Annotate the plot
plt.legend(('setosa', 'versicolor', 'virginica'), loc='lower right')
_ = plt.xlabel('petal length (cm)')
_ = plt.ylabel('ECDF')

# Display the plot
plt.show()

In [ ]:
# Comparing percentiles to ECDFs

# Compute percentiles

# Specify array of percentiles: percentiles
percentiles = np.array([2.5, 25,50,75, 97.5])

# Compute percentiles: ptiles_vers
ptiles_vers = np.percentile(versicolor_petal_length, percentiles)

# Print the result
print(ptiles_vers)

# Plot the ECDF
_ = plt.plot(x_vers, y_vers, '.')
_ = plt.xlabel('petal length (cm)')
_ = plt.ylabel('ECDF')

# Overlay percentiles as red diamonds.
_ = plt.plot(ptiles_vers, percentiles/100, marker='D', color='red',
         linestyle='none')

# Show the plot
plt.show()

In [ ]:
# Compare ditribution of feature with normal distribution

# Compute mean and standard deviation: mu, sigma
mu = np.mean(belmont_no_outliers)
sigma = np.std(belmont_no_outliers)

# Sample out of a normal distribution with this mu and sigma: samples
samples = np.random.normal(mu, sigma, 10000)

# Get the CDF of the samples and of the data
x_theor, y_theor = ecdf(samples)
x, y = ecdf(belmont_no_outliers)

# Plot the CDFs and show the plot
_ = plt.plot(x_theor, y_theor)
_ = plt.plot(x, y, marker='.', linestyle='none')
_ = plt.xlabel('Belmont winning time (sec.)')
_ = plt.ylabel('CDF')
plt.show()

In [ ]:
# Linear regression

# Plot the illiteracy rate versus fertility
_ = plt.plot(illiteracy, fertility, marker='.', linestyle='none')
plt.margins(0.02)
_ = plt.xlabel('percent illiterate')
_ = plt.ylabel('fertility')

# Perform a linear regression using np.polyfit(): a, b
a, b = np.polyfit(illiteracy, fertility, 1)

# Print the results to the screen
print('slope =', a, 'children per woman / percent illiterate')
print('intercept =', b, 'children per woman')

# Make theoretical line to plot
x = np.array([0, 100])
y = a * x + b

# Add regression line to your plot
_ = plt.plot(x, y)

# Draw the plot
plt.show()

In [ ]:
# Plotting bootstrap samples

for _ in range(50):
    # Generate bootstrap sample: bs_sample
    bs_sample = np.random.choice(rainfall, size=len(rainfall))

    # Compute and plot ECDF from bootstrap sample
    x, y = ecdf(bs_sample)
    _ = plt.plot(x, y, marker='.', linestyle='none',
                 color='gray', alpha=0.1)

# Compute and plot ECDF from original data
x, y = ecdf(rainfall)
_ = plt.plot(x, y, marker='.')

# Make margins and label axes
plt.margins(0.02)
_ = plt.xlabel('yearly rainfall (mm)')
_ = plt.ylabel('ECDF')

# Show the plot
plt.show()

In [ ]:
# Function to generate bootstrap replicates
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

# Take 10,000 bootstrap replicates of the mean: bs_replicates
bs_replicates = draw_bs_reps(rainfall, np.mean, 10000)

# Compute and print SEM
sem = np.std(rainfall) / np.sqrt(len(rainfall))
print(sem)

# Compute and print standard deviation of bootstrap replicates
bs_std = np.std(bs_replicates)
print(bs_std)

# Make a histogram of the results
_ = plt.hist(bs_replicates, bins=50, normed=True)
_ = plt.xlabel('mean annual rainfall (mm)')
_ = plt.ylabel('PDF')

# Show the plot
plt.show()

# Compute the 95% confidence interval: conf_int
conf_int = np.percentile(bs_replicates, [2.5, 97.5])

In [ ]:
# Pairs bootstrap

# Function
def draw_bs_pairs_linreg(x, y, size=1):
    """Perform pairs bootstrap for linear regression."""

    # Set up array of indices to sample from: inds
    inds = np.arange(0, len(x))

    # Initialize replicates: bs_slope_reps, bs_intercept_reps
    bs_slope_reps = np.empty(size)
    bs_intercept_reps = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_inds = np.random.choice(inds, size=len(inds))
        bs_x, bs_y = x[bs_inds], y[bs_inds]
        bs_slope_reps[i], bs_intercept_reps[i] = np.polyfit(bs_x, bs_y, 1)

    return bs_slope_reps, bs_intercept_reps


# Generate replicates of slope and intercept using pairs bootstrap
bs_slope_reps, bs_intercept_reps = draw_bs_pairs_linreg(illiteracy, fertility, size=1000)

# Compute and print 95% CI for slope
print(np.percentile(bs_slope_reps,[2.5, 97.5]))

# Plot the histogram
_ = plt.hist(bs_slope_reps, bins=50, normed=True)
_ = plt.xlabel('slope')
_ = plt.ylabel('PDF')
plt.show()

# Generate array of x-values for bootstrap lines: x
x = np.array([0,100])

# Plot the bootstrap lines
for i in range(0,100):
    _ = plt.plot(x, 
                 bs_slope_reps[i]*x + bs_intercept_reps[i],
                 linewidth=0.5, alpha=0.2, color='red')

# Plot the data
_ = plt.plot(illiteracy, fertility, marker='.', linestyle='none')

# Label axes, set the margins, and show the plot
_ = plt.xlabel('illiteracy')
_ = plt.ylabel('fertility')
plt.margins(0.02)
plt.show()

In [ ]:
# Generating permutation replicates

# Function
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

# EDA prior to hypothesis testing

# Make bee swarm plot
_ = sns.swarmplot(x="ID", y="impact_force", data=df)

# Label axes
_ = plt.xlabel('frog')
_ = plt.ylabel('impact force (N)')

# Show the plot
plt.show()


# Permutation test
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

# Compute difference of mean impact force from experiment: empirical_diff_means
empirical_diff_means = diff_of_means(force_a, force_b)

# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(force_a, force_b,
                                 diff_of_means, size=10000)

# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)

# Print the result
print('p-value =', p)